## Import necessary Python packages

In [1]:
import os
import pickle
import multirecording_spikeanalysis as spike
import numpy as np
# import pandas as pd # use pandas for more functionality
import modin.pandas as pd # use modin to speed things up

## Define working directory relative to repository

In [2]:
# get working directory as paerent directory of current directory
cwd = os.getcwd()
pwd = os.path.dirname(cwd)

## Import .pkl files

In [3]:
# import pickle files given by lab
with open(pwd + '/01_Raw_data/phase2_collection.pkl', 'rb') as f:
    phase2 = pickle.load(f)

# Not usoing phase 3 data
# with open(pwd + '/01_Raw_data/phase3_collection.pkl', 'rb') as f:
#     phase3 = pickle.load(f)

## Notes and reminders about data

In [4]:
# show the subject to which each recording belongs to
# .subject

# shows the type of unit/neuron (we only use good units in unit_timestamps)
# .labels_dict

# all the timestamps for all units
# .timestamps_var

# all the timestamps for each unit
# .unit_timestamps 

# the behaviour labels for each timestamp with a starting and ending time
# .event_dict

# Rework Data from .pkl files

### Behavioral labels and event ranges dataframe

In [5]:
# create metadata dataframe
metadata_df = pd.DataFrame() # create empty dataframe
temp_df_lst = [] # create empty list to store dataframes
for i, j in phase2.collection.items(): # loop through each recording
    temp_df = pd.DataFrame() # create empty dataframe
    start_time_lst = [] # create empty list to store start times
    end_time_lst = [] # create empty list to store end times
    behavior_lab_lst = [] # create empty list to store behavior labels
    for k, v in j.event_dict.items(): # loop through each behavior
        start_time_lst += list(v[:,0]) # add start times to list
        end_time_lst += list(v[:,1]) # add end times to list
        behavior_lab_lst += list([k] * len(v)) # add behavior labels to list
    temp_df['behavior_label'] = behavior_lab_lst # add behavior labels to dataframe
    temp_df['start_time'] = start_time_lst # add start times to dataframe
    temp_df['end_time'] = end_time_lst # add end times to dataframe
    temp_df['collection_key'] = i # add recording name to dataframe
    temp_df['subject'] = j.subject # add subject to dataframe
    temp_df_lst.append(temp_df) # add dataframe to list
metadata_df = pd.concat(temp_df_lst) # concatenate all dataframes in list to one dataframe
# show dataframe
# metadata_df
# save dataframe as csv
metadata_df.to_csv(pwd + "/02_Clean_data/recording_range_metadata.csv", index=False)

To remove this warning, run the following python code before doing dataframe operations:

    from distributed import Client

    client = Client()

Please refer to https://modin.readthedocs.io/en/stable/supported_apis/defaulting_to_pandas.html for explanation.


### Units dataframe & unit timestamps list
Save a dataframe of the units used for each subject at each recording session.
Extract the timestamps for each event for each unit from the data.
Merge metadata dataframes into a single long dataframe. So that metadata can be easily related to each recording session.

In [6]:
# create metadata dataframe
metadata_df = pd.DataFrame()
temp_df_lst = []
for i, j in phase2.collection.items():
    temp_df = pd.DataFrame()
    temp_df['units'] = j.unit_timestamps.keys() # add neurons/units to dataframe
    temp_df['timestamps'] = j.unit_timestamps.values() # add neurons/units to dataframe
    temp_df['collection_key'] = i # add recording name to dataframe
    temp_df['subject'] = j.subject # add subject to dataframe
    temp_df_lst.append(temp_df) # add dataframe to list
metadata_df = pd.concat(temp_df_lst) # concatenate all dataframes in list to one dataframe
# show dataframe
metadata_df
# save dataframe as csv
metadata_df.to_csv(pwd + "/02_Clean_data/units_data.csv", index=False)